In [ ]:
#modeling


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from statsmodels.tsa.arima.model import ARIMA


In [ ]:
df = pd.read_csv("data_no_outliers.csv")
df.info()

In [ ]:
#let's build time based DF's
df['Txn_date'] = pd.to_datetime(df['Txn_date'])
df['Year'] = df['Txn_date'].dt.year
df['Month'] = df['Txn_date'].dt.month
df['Day'] = df['Txn_date'].dt.day
#day of the week and weekends
df['DayOfWeek'] = df['Txn_date'].dt.dayofweek
df['Weekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

In [ ]:
#train and test
X_train, X_test, y_train, y_test = train_test_split(df[['Year', 'Month', 'Day', 'DayOfWeek', 'Weekend']], df['Total_Amt'], test_size=0.2, random_state=0)


In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print('RMSE: ', rmse)

In [ ]:
r2 = r2_score(y_test, predictions)
print('R2: ', r2)


In [ ]:
#negative R2 is unacceptable 

In [ ]:
df_ts = df.resample('M', on='Txn_date')['Total_Amt'].sum()


In [ ]:
model = ARIMA(df_ts, order=(5,1,0))
model_fit = model.fit(disp=0)

plt.figure(figsize=(15,10))
model_fit.plot_predict(dynamic=False)
plt.show()

In [ ]:
model = ARIMA(df_ts['Total_Amt'], order=(5,1,0)) 
model_fit = model.fit(disp=0)


fig, ax = plt.subplots()
ax.plot(df_ts['Total_Amt'], label='original')
ax.plot(model_fit.predict(), label='forecast')
plt.legend()
plt.show()

In [ ]:
model_fit = model.fit()


In [ ]:
print(model_fit.summary())


In [ ]:
plt.figure(figsize=(15,10))
model_fit.plot_predict(dynamic=False)
plt.show()
